In [56]:
from google.colab import drive
drive.mount('/content/drive')
data_root ='/content/drive/My Drive/Chatbot'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [57]:
# importing relevant libraries

import json
import string
import random

import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
nltk.download("punkt")
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [58]:
#Loading the Dataset: intens.json

data_file = open(data_root + '/Percakapan.json').read()
data = json.loads(data_file)
data

{'Percakapan': [{'tag': 'Hai kak',
   'patterns': ['Hallo Kak', 'Selamat Pagi Ka'],
   'responses': ['Hai kak, apa ada yang bisa dibantu',
    'Selamat Pagi kak, apa ada yang bisa dibantu',
    'Hallo kak, apa ada yang bisa dibantu'],
   'context': ''},
  {'tag': 'Assalamualaikum teman',
   'patterns': [],
   'responses': ['Waalaikumsalam teman'],
   'context': ['']},
  {'tag': 'Halo bro',
   'patterns': [],
   'responses': ['Apa Bro', 'Kenapa Bro'],
   'context': ['']},
  {'tag': 'woi bang',
   'patterns': [],
   'responses': ['yang sopan dong',
    'nyapa orang masa begitu',
    'pernah diajarin sopan santun ga sih'],
   'context': ['']},
  {'tag': 'tidak ada jawaban',
   'patterns': [],
   'responses': ['Sorry, saya tidak tahu maksud anda',
    'Berikan saya info lebih'],
   'context': ['']}]}

In [59]:
# Extracing data_x (features) and data y (target)

words = [] #For bow model/ vocabulary for patterns
classes = [] #For bow model/ vocabulary for tags
data_x = [] #For storing each pattern
data_y = [] #For storing tag corresponding to each pattern in data_X
#Iterating over all the intents

for intent in data ["Percakapan"]:
  for pattern in intent["patterns"]:
    tokens = nltk.word_tokenize(pattern) # tokenize each pattern
    words.extend(tokens) #and append tokens to word
    data_x.append(pattern) #appending pattern to data_X
    data_y.append(intent["tag"]), #appending the associated tag to each pattern

    #adding the tag to the classes if it's not there already
    if intent["tag"] not in classes:
      classes.append(intent["tag"])

#initializing lemmatizer to get stem of words
lemmatizer = WordNetLemmatizer()

word = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]

word = sorted(set(words))
classes = sorted(set(classes))
classes

['Hai kak']

In [60]:
#text to numbers

training = []
out_empty = [0] * len(classes)

#creating the bag of words model

for idx, doc in enumerate(data_x):
  bow = []
  text = lemmatizer.lemmatize(doc.lower())
  for word in words:
    bow.append(1) if word in text else bow.append(0)

  #mark the index of class that the current pattern is associated
  #to

  output_row = list(out_empty)
  output_row[classes.index(data_y[idx])]

  #add the one hot encoded Bow and associated classes training

  training.append ([bow, output_row])

#shuffle the data and convert it to an array

random.shuffle(training)
training = np.array(training, dtype=object)

#split the features and target labels

train_x = np.array(list(training[:, 0]))
train_y = np.array(list(training[:,1]))

train_x

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])

In [61]:
model = Sequential ()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation = "softmax"))
adam = tf.keras.optimizers.legacy.Adam(learning_rate=0.01, decay=1e-6)
model.compile(loss = "categorical_crossentropy",
              optimizer=adam,
              metrics =['accuracy'])
print(model.summary())
model.fit(x=train_x, y = train_y, epochs=500, verbose=1)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 128)               768       
                                                                 
 dropout_6 (Dropout)         (None, 128)               0         
                                                                 
 dense_10 (Dense)            (None, 64)                8256      
                                                                 
 dropout_7 (Dropout)         (None, 64)                0         
                                                                 
 dense_11 (Dense)            (None, 1)                 65        
                                                                 
Total params: 9089 (35.50 KB)
Trainable params: 9089 (35.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/500


/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/dispatch.py:1176: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


1/1 [==============================] - 0s 490ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 2/500
1/1 [==============================] - 0s 9ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 3/500
1/1 [==============================] - 0s 9ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 4/500
1/1 [==============================] - 0s 8ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 5/500
1/1 [==============================] - 0s 13ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 6/500
1/1 [==============================] - 0s 10ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 7/500
1/1 [==============================] - 0s 9ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 8/500
1/1 [==============================] - 0s 10ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 9/500
1/1 [==============================] - 0s 10ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 10/500
1/1 [==============================] - 0s 9ms/st

In [70]:
def clean_text(text):
  tokens = nltk.word_tokenize(text)
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  return tokens

def bag_of_words(text, vocab):
  tokens = clean_text(text)
  bow = [0] * len(vocab)
  for w in tokens:
    for idx, word in enumerate(vocab):
      if word == w:
        bow[idx] = 1
  return np.array(bow)

def pred_class(text, vocab, labels):
  bow = bag_of_words(text, vocab)
  result = model.predict(np.array([bow]))[0] #Extracting probabilities
  thresh = 0.5
  y_pred = [[indx, res] for indx, res in enumerate(result) if res > thresh]
  y_pred.sort(key=lambda x: x[1], reverse=True) #Sorting by values of probability in decreasing order
  return_list = []
  for r in y_pred:
    return_list.append(labels[r[0]]) #Contains labels(tags) for highest probability
  return return_list

def get_response(Percakapan_list, Percakapan_json):
  if len(Percakapan_list) == 0:
    result = "Sorry! I don't understand."
  else:
    tag = Percakapan_list[0]
    list_of_intents = Percakapan_json["Percakapan"]
    for i in list_of_intents:
      if i["tag"] == tag:
        result = random.choice(i["responses"])
        break
  return result

In [71]:
# Interacting with the chatbot
print("Press 0 if you don't want to chat with our ChatBot.")
while True:
    message = input("")
    if message == "0":
      break
    intents = pred_class(message, words, classes)
    result = get_response(Percakapan, data)
    print(result)

Press 0 if you don't want to chat with our ChatBot.
Hai kak
1/1 [==============================] - 0s 20ms/step


NameError: ignored